In [1]:
pip install arviz

     |████████████████████████████████| 1.6MB 8.8MB/s 
     |████████████████████████████████| 768kB 34.6MB/s 
     |████████████████████████████████| 4.7MB 48.3MB/s 
     |████████████████████████████████| 317kB 39.7MB/s 
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1


In [2]:
import pystan
import numpy as np
import matplotlib.pyplot as plt
import arviz
import pickle
import cmdstanpy
import json
import pandas as pd

In [3]:
with open('/content/drive/MyDrive/TFM-Crowd learning/multinomial.json') as json_file:
    data = json.load(json_file)

In [4]:
dades={'Tasca': data['t_A'], 'Worker': data['w_A'], 'Anotació': data['ann'] }
df = pd.DataFrame(data=dades)
df

,Tasca,Worker,Anotació
0,0,14,2
1,0,13,2
2,0,4,2
3,0,2,2
4,0,7,1
...,...,...,...
9995,999,17,2
9996,999,2,2
9997,999,13,2
9998,999,14,2


In [5]:
sm_general = pystan.StanModel(file='/content/drive/MyDrive/TFM-Crowd learning/Fitxers STAN/general_dawid-skene.stan')


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b3b91d5d08ea08e32e8380bfec8b2a00 NOW.


In [6]:
  w = len(np.unique(df['Worker']))
  t = len(np.unique(df['Tasca']))
  a = len(df['Anotació'])
  k = len(np.unique(df['Anotació']))
  t_A = np.array(df['Tasca'])+1
  w_A = np.array(df['Worker'])+1
  ann = np.array(df['Anotació'])+1
  tau_prior = np.array([1,1,1])
  pi_prior = np.ones((k,k))

  dades_sm_gen = {'w':w,'t':t,'a':a,'k':k,'t_A':t_A,'w_A':w_A,'ann':ann,'tau_prior':tau_prior,'pi_prior':pi_prior} 

In [8]:
optim = sm_general.optimizing(data=dades_sm_gen)
optim

OrderedDict([('tau', array([0.59038748, 0.11256674, 0.29704578])),
             ('pi', array([[[0.14765941, 0.09129486, 0.76104573],
                      [0.85035189, 0.07909009, 0.07055802],
                      [0.21908585, 0.5887446 , 0.19216954]],
              
                     [[0.13058057, 0.11073535, 0.75868408],
                      [0.65408697, 0.12655529, 0.21935774],
                      [0.30767889, 0.51726566, 0.17505545]],
              
                     [[0.07408427, 0.10849884, 0.81741689],
                      [0.62412475, 0.08468081, 0.29119444],
                      [0.2137375 , 0.60222612, 0.18403639]],
              
                     [[0.09577564, 0.13826285, 0.76596151],
                      [0.69210761, 0.09324223, 0.21465016],
                      [0.19747432, 0.63426307, 0.16826261]],
              
                     [[0.08058212, 0.12262296, 0.79679492],
                      [0.55522348, 0.16270528, 0.28207124],
                      [

In [9]:
def correct_label_switching(sm,dades_sm,k):
  opt = sm.optimizing(data=dades_sm)

  acumulacio_permutacions=[]
  matrices_pi = opt['pi']
  tau = opt['tau']
  t_C = opt['t_C']
  log_p_t_C = opt['log_p_t_C']
  new_pi = np.zeros((len(matrices_pi),k,k))
  new_tau = np.zeros(k)
  new_t_C = np.zeros((len(t_C),k))
  new_log_p_t_C = np.zeros((len(log_p_t_C),k))

  for w in range(len(matrices_pi)):
    on_hauria_danar = np.argmax(matrices_pi[w],axis=1)
    acumulacio_permutacions.append(on_hauria_danar)
    for i in range(len(matrices_pi[w])): #aquest bucle corregeix les matrius pi
      new_pi[w][on_hauria_danar[i]] = matrices_pi[w][i]
  #amb aquesta part del codi determinem la permutació 'guanyadora' per aplicar-li a tau i t_C
  values, counts = np.unique(acumulacio_permutacions,axis=0, return_counts=True)
  ind = np.argmax(counts)
  permutacio = values[ind]
  #actualitzem tau, t_C i log_P_t_C
  for i in range(len(tau)):
    new_tau[permutacio[i]] = tau[i]

  for t in range(len(opt['t_C'])):
      for i in range(k):
        new_t_C[t][permutacio[i]] = t_C[t][i]
        new_log_p_t_C[t][permutacio[i]] = log_p_t_C[t][i]
  return new_tau,new_pi,new_t_C,new_log_p_t_C,permutacio





In [10]:
new_tau,new_pi,new_t_C,new_log_p_t_C, permutacio = correct_label_switching(sm_general,dades_sm_gen,k)

In [12]:
new_tau,new_pi,new_t_C,new_log_p_t_C, permutacio

(array([0.11254964, 0.29708801, 0.59036234]),
 array([[[0.85034893, 0.07908165, 0.07056942],
         [0.21908854, 0.58874451, 0.19216695],
         [0.14765338, 0.09130377, 0.76104285]],
 
        [[0.65434097, 0.12649401, 0.21916502],
         [0.30768034, 0.51723294, 0.17508672],
         [0.13057921, 0.11072228, 0.75869852]],
 
        [[0.62406019, 0.08483056, 0.29110925],
         [0.21375401, 0.60218972, 0.18405627],
         [0.07408611, 0.10848875, 0.81742514]],
 
        [[0.69214095, 0.09321926, 0.21463979],
         [0.19748926, 0.63428012, 0.16823062],
         [0.09577841, 0.13824472, 0.76597686]],
 
        [[0.55515051, 0.16258394, 0.28226555],
         [0.22260837, 0.6307409 , 0.14665073],
         [0.08058333, 0.12261333, 0.79680335]],
 
        [[0.59559594, 0.09822207, 0.30618198],
         [0.20057249, 0.5774031 , 0.22202442],
         [0.0902011 , 0.09381199, 0.81598691]],
 
        [[0.64338345, 0.15136375, 0.20525281],
         [0.16935728, 0.60493495, 0.2257077